In [3]:
!wget http://konect.cc/files/download.tsv.twin.tar.bz2

--2025-06-30 15:41:02--  http://konect.cc/files/download.tsv.twin.tar.bz2
Resolving konect.cc (konect.cc)... 138.48.20.248
Connecting to konect.cc (konect.cc)|138.48.20.248|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 79577 (78K) [application/x-bzip2]
Saving to: ‘download.tsv.twin.tar.bz2’

download.tsv.twin.t 100%[===================>]  77.71K   418KB/s    in 0.2s    

2025-06-30 15:41:02 (418 KB/s) - ‘download.tsv.twin.tar.bz2’ saved [79577/79577]



In [4]:
!tar -xvjf download.tsv.twin.tar.bz2

twin/
twin/out.twin
twin/README.twin
twin/meta.twin


In [1]:
import requests
from bs4 import BeautifulSoup

In [3]:
soup = BeautifulSoup(requests.get("http://en.sistercity.info/countries/").content)
print(soup.prettify())

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Strict//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-strict.dtd">
<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <title>
   Country index — sistercity.info
  </title>
  <meta content="width=device-width" name="viewport"/>
  <link href="/assets/css/stylesheet.css?f" rel="stylesheet" type="text/css"/>
  <link href="/favicon.ico?f" rel="shortcut icon"/>
  <script async="async" src="http://pagead2.googlesyndication.com/pagead/js/adsbygoogle.js">
  </script>
  <script src="/assets/js/common.js?f" type="text/javascript">
  </script>
 </head>
 <body>
  <div class="header-wrapper">
   <div class="header">
    <a href="/">
     Sister Cities of the World
    </a>
   </div>
  </div>
  <div class="wrapper">
   <div class="breadcrumbs">
    <a href="/">
     Home
    </a>
    <span class="unicode-arrow">
     ▶
    </span>
    Countries
   </div>
   <div

In [4]:
links = [(d.a.text, d.a['href']) for d in soup.find("div", attrs={"class":"content"}).find_all("div", attrs={"class":"label"})]
links

[('Afghanistan', '/countries/af.html'),
 ('Albania', '/countries/al.html'),
 ('Algeria', '/countries/dz.html'),
 ('Angola', '/countries/ao.html'),
 ('Argentina', '/countries/ar.html'),
 ('Armenia', '/countries/am.html'),
 ('Australia', '/countries/au.html'),
 ('Austria', '/countries/at.html'),
 ('Azerbaijan', '/countries/az.html'),
 ('Bahamas', '/countries/bs.html'),
 ('Bangladesh', '/countries/bd.html'),
 ('Barbados', '/countries/bb.html'),
 ('Belarus', '/countries/by.html'),
 ('Belgium', '/countries/be.html'),
 ('Belize', '/countries/bz.html'),
 ('Bolivia', '/countries/bo.html'),
 ('Bosnia and Herzegovina', '/countries/ba.html'),
 ('Botswana', '/countries/bw.html'),
 ('Brazil', '/countries/br.html'),
 ('Bulgaria', '/countries/bg.html'),
 ('Burkina Faso', '/countries/bf.html'),
 ('Cambodia', '/countries/kh.html'),
 ('Cameroon', '/countries/cm.html'),
 ('Canada', '/countries/ca.html'),
 ('Cape Verde', '/countries/cv.html'),
 ('Chile', '/countries/cl.html'),
 ('China', '/countries/cn.ht

In [5]:
test_soup = BeautifulSoup(requests.get("http://en.sistercity.info"+links[0][1]).content)
cities = [(c.a.text, c.a['href']) for c in test_soup.find("div", attrs={"class":"content"}).find_all("div", attrs={"class":"link-list-elem"})]
cities

[('Balkh', '/sister-cities/Balkh.html'),
 ('Bamyan', '/sister-cities/Bamyan.html'),
 ('Ghazni', '/sister-cities/Ghazni.html'),
 ('Kabul', '/sister-cities/Kabul.html'),
 ('Paghman', '/sister-cities/Paghman.html')]

In [6]:
test_soup_city = BeautifulSoup(requests.get("http://en.sistercity.info"+cities[0][1]).content)
sis_cities = [d.text.strip().split("\xa0")[0] for d in test_soup_city.find("div", attrs={"class":"content"}).find_all("div", attrs={"class":"label"})]
sis_cities

['Bukhara']

In [12]:
def get_sis_for_one(url):
    soup = BeautifulSoup(requests.get("http://en.sistercity.info"+url).content)
    return [d.text.strip().split("\xa0")[0] for d in soup.find("div", attrs={"class":"content"}).find_all("div", attrs={"class":"label"})]

def get_cities_for_one_country(url):
    soup = BeautifulSoup(requests.get("http://en.sistercity.info"+url).content)
    return [(c.a.text, c.a['href']) for c in soup.find("div", attrs={"class":"content"}).find_all("div", attrs={"class":"link-list-elem"})]

In [22]:
from tqdm import tqdm
import pandas as pd

up_to = pd.read_csv("sis_cities_to_switz.csv")
up_to = up_to.rename(columns={"1":"city"})

data = []
for country, url in tqdm(links):
    cities = get_cities_for_one_country(url)
    for city, c_url in cities:
        if city not in up_to['city'].to_list():
            try:
                sisses = get_sis_for_one(c_url)
                for sis in sisses:
                    data.append((country, city, sis))
            except Exception as e:
                print(e)
                continue

  0%|          | 0/165 [00:00<?, ?it/s]

 86%|████████▌ | 142/165 [00:33<00:05,  4.38it/s]

'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'
'NoneType' object has no attribute 'find_all'


100%|██████████| 165/165 [08:52<00:00,  3.22s/it] 


In [23]:
import pandas as pd

df = pd.DataFrame(data)
df

,0,1,2
0,Switzerland,Binn,Arbon
1,Switzerland,Binn,Urtenen-Schönbühl
2,Switzerland,Birsfelden,Plan-les-Ouates
3,Switzerland,Bischofszell,Battaglia Terme
4,Switzerland,Bischofszell,Tuttlingen
...,...,...,...
6875,Zimbabwe,Harare,Munich
6876,Zimbabwe,Harare,Nottingham
6877,Zimbabwe,Mutare,Haarlem
6878,Zimbabwe,Mutare,"Portland, Oregon"


In [28]:
up_to = up_to[['0', 'city', '2']].rename(columns={"0":"country", "2":"sister"})
up_to

,country,city,sister
0,Afghanistan,Balkh,Bukhara
1,Afghanistan,Bamyan,"Gering, Nebraska"
2,Afghanistan,Ghazni,"Hayward, California"
3,Afghanistan,Kabul,Dushanbe
4,Afghanistan,Kabul,Istanbul
...,...,...,...
18932,Switzerland,Bern,Tyumen
18933,Switzerland,Bern,Vienna
18934,Switzerland,Bettingen,Safien
18935,Switzerland,Bettlach,Lütschental


In [31]:
full = pd.concat([up_to, df.rename(columns={0:"country",1:"city", 2:"sister"})])

In [32]:
full

,country,city,sister
0,Afghanistan,Balkh,Bukhara
1,Afghanistan,Bamyan,"Gering, Nebraska"
2,Afghanistan,Ghazni,"Hayward, California"
3,Afghanistan,Kabul,Dushanbe
4,Afghanistan,Kabul,Istanbul
...,...,...,...
6875,Zimbabwe,Harare,Munich
6876,Zimbabwe,Harare,Nottingham
6877,Zimbabwe,Mutare,Haarlem
6878,Zimbabwe,Mutare,"Portland, Oregon"


In [33]:
full.to_csv("sister_cities.csv")

In [38]:
import networkx as nx

G = nx.Graph()
for i, row in full.iterrows():
    G.add_node(row["city"], country=row['country'])
    G.add_node(row['sister'])
    G.add_edge(row['city'], row["sister"])

In [39]:
len(G)

12696

In [61]:
import random

n = 6
start_node = random.choice(list(G.nodes()))
subgraph_nodes = {start_node}

current = start_node
while len(subgraph_nodes) < n:
    candidates = set()
    for node in subgraph_nodes:
        candidates.update(G.neighbors(node))
    candidates -= subgraph_nodes
    if not candidates:
        break
    new_node = random.choice(list(candidates))
    subgraph_nodes.add(new_node)

subgraph_nodes

{'Ibbenbüren',
 'Jastrzębie-Zdrój',
 'Kuşadası',
 'Novomoskovsk',
 'Prievidza',
 'Tourcoing'}